In [1]:
import cv2
import os
from PIL import Image
import numpy as np
path = './dataset/'
no_tumor = os.listdir(path+'no/')
tumor = os.listdir(path+'yes/')
dataset = []
label = []
for i,image_name in enumerate(no_tumor):
    if(image_name.split('.')[1]=='jpg'):
        image = cv2.imread(path+'no/'+image_name) # READ IMAGE IN BGR FORMAT
        image = Image.fromarray(image,'RGB') #cONVERT FROM BGR TO RGB FORMAT BECAUSE MANY LIBRARIES EXPECT IN RGB FORMAT
        image = image.resize((64,64))
        dataset.append(np.array(image))
        label.append(0)
for i,image_name in enumerate(tumor):
    if(image_name.split('.')[1]=='jpg'):
        image = cv2.imread(path+'yes/'+image_name)
        image = Image.fromarray(image,'RGB')
        image = image.resize((64,64))
        dataset.append(np.array(image))
        label.append(1)


In [2]:
print(len(dataset))
print(len(label))

3000
3000


In [3]:
dataset = np.array(dataset)
label = np.array(label)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
x_train,x_test,y_train,y_test = train_test_split(dataset,label,test_size=0.2,random_state=0)

In [6]:
print(x_train.shape)

(2400, 64, 64, 3)


In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense
from keras.utils import to_categorical

In [8]:
#NORMALISE DATA
x_train = normalize(x_train,axis=1)
x_test = normalize(x_test,axis=1)



In [9]:
y_train = to_categorical(y_train,num_classes=2)
y_test = to_categorical(y_test,num_classes=2)

In [10]:
INPUT_SIZE = 64

In [11]:
#BUILD MODEL
model = Sequential()
model.add(Conv2D(32,(3,3),input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(32,(3,3),kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))




In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.fit(x_train,y_train,batch_size=16,verbose=1,epochs=10,validation_data=(x_test,y_test),shuffle=False)

Epoch 1/10
150/150 [==============================] - 10s 58ms/step - loss: 0.5562 - accuracy: 0.7254 - val_loss: 0.4528 - val_accuracy: 0.7883
Epoch 2/10
150/150 [==============================] - 9s 62ms/step - loss: 0.4152 - accuracy: 0.8283 - val_loss: 0.4076 - val_accuracy: 0.8083
Epoch 3/10
150/150 [==============================] - 10s 66ms/step - loss: 0.3281 - accuracy: 0.8621 - val_loss: 0.3173 - val_accuracy: 0.8517
Epoch 4/10
150/150 [==============================] - 10s 64ms/step - loss: 0.2405 - accuracy: 0.9029 - val_loss: 0.2045 - val_accuracy: 0.9200
Epoch 5/10
150/150 [==============================] - 11s 71ms/step - loss: 0.1785 - accuracy: 0.9337 - val_loss: 0.1561 - val_accuracy: 0.9533
Epoch 6/10
150/150 [==============================] - 11s 74ms/step - loss: 0.1308 - accuracy: 0.9571 - val_loss: 0.1161 - val_accuracy: 0.9617
Epoch 7/10
150/150 [==============================] - 9s 63ms/step - loss: 0.0886 - accuracy: 0.9708 - val_loss: 0.1262 - val_accuracy: 0

In [14]:
model.save('BrainTumor.h5')

C:\Users\PRADEEP_HILTRACK\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# MODEL EVALUATION 

In [15]:
from sklearn.metrics import confusion_matrix, classification_report

In [16]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

19/19 [==============================] - 1s 28ms/step


In [17]:
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

classification_rep = classification_report(y_true, y_pred_classes)
print("Classification Report:")
print(classification_rep)

Confusion Matrix:
[[328  15]
 [  5 252]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       343
           1       0.94      0.98      0.96       257

    accuracy                           0.97       600
   macro avg       0.96      0.97      0.97       600
weighted avg       0.97      0.97      0.97       600

